In [66]:
# Import for the Assigment3
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import math
import pickle
import copy as copy
from numpy import random
from tqdm import tqdm

In [67]:
# Global Variable
training_file = "data_batch_1"
validation_file = "data_batch_2"
test_file = "test_batch"
data_folder = "../../save/assigment1-save/Datasets/cifar-10-batches-py/"
N = 10000
K = 10
d = 3072
m = 50

In [138]:
# Trasnform a vector of label to a vector of hot-one(dumies)
def hot_one(y):
    Y = []
    for yi in y:
        yihot = [0] * K
        yihot[yi] = 1
        Y.append(yihot)
    return np.array(Y)
    
    
#Functions that load data from file.
def load_batch(file_name):

    with open(file_name, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    y = np.hstack(dict[b'labels'])
    X = dict[b'data'].astype(float).T / 255.0
    Y = hot_one(y).T
    return X,Y,y

#Function that normalize the data( Only mean)
#Input : - X(d,N)
#Output: - X(d,N)

def normalizeData(X):
    m = np.mean(X,1)
    std = np.std(X,1)
    #X = (X.T - m).T
    return X, m , std

def plot_curve(lostTrain, lostVal):
    rr = range(len(lostVal))
    plt.plot(rr, lostVal,'green')
    plt.plot(rr, lostTrain,'blue')
    plt.title("loss during the training for eta = " + str(eta) + " lambda = " + str(lamb))
    green_patch = mpatches.Patch(color='blue', label="Training loss")
    blue_patch = mpatches.Patch(color='green', label="Validation loss")
    plt.legend(handles=[green_patch, blue_patch])
    plt.savefig(str(lamb) + '_'+ str(eta) + '.png')
    plt.show()
    
def initializeWb_Hu(dim_list):
    mu = 0
    W = []
    b = []
    for i in range(len(dim_list) - 1):
        W.append(np.random.normal(mu, np.sqrt(2.0 / dim_list[i + 1]), (dim_list[i + 1], dim_list[i])))
        b.append(np.random.normal(mu, np.sqrt(2.0 / dim_list[i + 1]), (dim_list[i + 1], 1)))
    return W, b
def initializeWb_Random(dim_list):
    mu = 0
    W = []
    b = []
    for i in range(len(dim_list) - 1):
        W.append(np.random.normal(mu, 0.1, (dim_list[i + 1], dim_list[i])))
        b.append(np.random.normal(mu, 0.1, (dim_list[i + 1], 1)))
    return W, b

In [139]:
#Load of every datasete
X_train, Y_train, y_train = load_batch(data_folder + training_file)
X_test, Y_test, y_test = load_batch(data_folder + test_file)
X_val, Y_val, y_val = load_batch(data_folder + validation_file)

X_train, m, std = normalizeData(X_train)
#X_test = (X_test.T - m).T
#X_val = (X_val.T - m).T

In [370]:
def batch_norm(sll, mul, nul, eps):
    assert(len(nul) == len(mul))
    sl = copy.deepcopy(sll.T)
    for i in range(len(sl)):
        sl[i] = (sl[i] - mul) * ((nul + eps) ** -0.5)
        assert(len(sl[i]) == len(mul))
    return  sl.T


def evaluate_Classifier_batch(X, W, b, eps, mean_layer=None, std_layer=None):
    X_i = []
    h = X
    X_i.append(copy.deepcopy(h))
    X_i_hat = []
    mean_layer_out = []
    std_layer_out = []
    X_i_hat.append(copy.deepcopy(h))
    
    for i in range(0,len(W) - 1):
  
        s1 = np.dot(W[i], h) + b[i]
        X_i.append(copy.deepcopy(s1))
        if(mean_layer == None): # train
            mul = np.mean(s1, 1)
            nul = np.std(s1, 1)
            mean_layer_out.append(mul)
            std_layer_out.append(nul)
            s_norm = batch_norm(s1, mul, nul, eps)
        else:
            print(s1)
            print(eps)
            s_norm = batch_norm(s1, mean_layer[i], std_layer[i], eps)
        h = np.maximum(s_norm, 0)
        X_i_hat.append(copy.deepcopy(s_norm))

    s = np.dot(W[-1], h) + b[-1]
    #X_i.append(copy.deepcopy(s))
    aux = np.exp(s)
    summ = np.sum(aux, axis=1) #axis=0 sum by colum so we got 1000 therm to normalize
    p = aux / summ
    return p, X_i, X_i_hat, mean_layer_out, std_layer_out
def compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer=None, std_layer=None, p=None):
    J = 0.0
    if(np.all(p == None)):
        p, _, _, _, _ = evaluate_Classifier_batch(X, W, b, eps, mean_layer, std_layer)
    for index in range(X.shape[1]):
        J += - np.log(np.dot(p[:,index], Y[:,index]))
    J /= X.shape[1]
    for w in W:
        J += lamb * np.sum(np.square(w))
    return J
    
def compute_Accuracy_batch(X, y, W, b, eps, mean_layer=None, std_layer=None, p=None):
    if(np.all(p == None)):
        p, _, _, _, _= evaluate_Classifier_batch(X, W, b, eps, mean_layer, std_layer) 
    pred = np.argmax(p,axis=0)
    acc = 0.0
    for i in range(len(pred)):
        if(pred[i] == y[i]):
            acc += 1
    return acc / len(pred)

## Define Gradient

In [371]:
def compute_Gradients_Num_batch(X, Y, P, W, b, lamb, h, eps, mean_layer=None, std_layer=None):
    print(W[0].shape)
    grad_W = []
    grad_b = []
    N = X.shape[1]
    # Initialization of grad
    for i in range(len(W)):
        grad_W.append(np.zeros((W[i].shape[0],W[i].shape[1])))
        grad_b.append(np.zeros((b[i].shape[0],b[i].shape[1])))

    c =  compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer)
    #print(c)
    for index in range(len(W)):
        for i in range(W[index].shape[0]):
            for j in range(W[index].shape[1]):
                W[index][i][j] += h
                c2 = compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer)
                W[index][i][j] -= 2 * h
                c1 = compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer)
                grad_W[index][i][j] = (c2 - c1) / (2 * h)
                W[index][i][j] += h

        for i in range(len(b[index])):
            b[index][i] += h
            c2 = compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer)
            b[index][i] -= 2 * h
            c1 = compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer)
            b[index][i] += h
            grad_b[index][i] = (c2 - c1) / (2 * h)
            
                
   
    return np.array(grad_W), np.array(grad_b)

In [1]:

def compute_Gradients_batch(X, Y, p, X_i, X_i_hat, W, b, lamb, mean_layer, std_layer):
    grad_W = []
    grad_b = []
    N = X.shape[1]
    #print(N)
    # Initialization of grad
    for i in range(len(W)):
        grad_W.append(np.zeros((W[i].shape[0],W[i].shape[1])))
        grad_b.append(np.zeros((b[i].shape[0],b[i].shape[1])))
    
    g = - (Y.T - p.T).T
    
    gx = np.dot(g, np.maximum(X_i_hat[-1].T, 0))
    
    grad_b[-1] += np.mean(g, axis=-1, keepdims=True)
    grad_W[-1] += gx
    grad_W[-1] /= N
    grad_W[-1] += 2 * lamb * W[-1]
    
    #First update of g
    
    g = np.dot(g.T,W[-1])
    g1 = np.where(X_i_hat[-1] > 0, 1, 0) 
    g = np.multiply(g.T, g1)
    for l in range(len(W) - 2, -1, -1):
        g = g.T
        X_i_centered = np.subtract(X_i[l + 1].T, mean_layer[l])
        
        gradJ_std = - (std_layer[l] ** (-1.5)) * np.sum(g * X_i_centered, axis=1, keepdims=True)
        gradJ_mean = - (std_layer[l] ** (-0.5)) * np.sum(g, axis=1, keepdims=True)
        g =  g * (std_layer[l] ** (-0.5)) + (X_i_centered * gradJ_std) / N #+ (gradJ_mean / N)
         
        g = g.T
        grad_b[l] += np.mean(g, axis=1, keepdims=True)
        
        gx = np.dot(g,  np.maximum(X_i_hat[l].T, 0))
        grad_W[l] += gx
        grad_W[l] /= N
        grad_W[l] += 2 * lamb * W[l]
        
        g = np.dot(g.T,W[l])
        g1 = np.where(X_i_hat[l] > 0, 1, 0)
        g = np.multiply(g.T, g1)
        
        
        
        
    return grad_W, grad_b



In [2]:
K = 10
d = 5
n = 10
dim_list = [d, 20, 5, 7,K]
h = 1e-4
lamb = 0
rho = 0.9

eps = 1e-06

X = X_train[:d,:n]
Y = Y_train[:,:n]
W_init, b_init = initializeWb_Hu(dim_list)

p, X_i,X_i_hat, mean_layer, std_layer  = evaluate_Classifier_batch(X, W_init, b_init , eps)

grad_W11, grad_b11  = compute_Gradients_Num_batch(X, Y, p, W_init, b_init, lamb, h, eps)
grad_W1, grad_b1 = compute_Gradients_batch(X, Y, p, X_i, X_i_hat, W_init, b_init, lamb, mean_layer, std_layer)

for i in range(len(W_init)):
    print("Norm for W & b for the " + str(i) + "layer")
    assert(grad_W1[i].shape ==grad_W11[i].shape)
    print(np.linalg.norm(grad_W1[i] - grad_W11[i],ord=1) / max(h ,np.linalg.norm(grad_W1[i] ,ord=1) + np.linalg.norm(grad_W11[i] ,ord=1)))
    print(np.linalg.norm(grad_b1[i] - grad_b11[i],ord=1) / max(h ,np.linalg.norm(grad_b1[i] ,ord=1) + np.linalg.norm(grad_b11[i] ,ord=1)))

NameError: name 'X_train' is not defined

In [284]:
print(grad_b11[0])
print(grad_b1[0])

[[0.]
 [0.]]
[[-0.12060852]
 [-0.10509788]]


In [365]:
def fit(X, Y, y, X_val, Y_val, y_val, lamb, n_batch, rho, eta , n_epochs, dim_list, W, b, eps, norm_coef):
    d = X.shape[0]
    N = X.shape[1]
    print(W[0].shape)
    print(b[0].shape)
    lostTrain = []
    lostVal = []
    W_momentum = []
    b_momentum = []
    for i in range(len(W)):
        W_momentum.append(np.zeros((W[i].shape[0],W[i].shape[1])))
        b_momentum.append(np.zeros((b[i].shape[0],b[i].shape[1])))
    mean_layer=None
    std_layer=None
    for _ in range(n_epochs):
        
        #print(random.permutation(range(X.shape[1])))
        for j in range(int(N / n_batch)):
            print('j' , j)
            start = j * n_batch
            end = (j + 1) * n_batch
            X_batch = X[:,start:end]
            Y_batch = Y[:,start:end]

            p, X_i, X_i_hat, mean_layer_out, std_layer_out = evaluate_Classifier_batch(X_batch, W, b, eps)
            #print(compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer_out, std_layer_out))
            #print(mean_layer_out)

            if(mean_layer == None or std_layer == None):
                mean_layer = copy.deepcopy(mean_layer_out)
                std_layer = copy.deepcopy(std_layer_out)

            grad_W, grad_b = compute_Gradients_Num_batch(X_batch, Y_batch, p, W, b, lamb, 10e-04,mean_layer_out, std_layer_out)
            #(X, Y, P, W, b, lamb, h, eps, mean_layer=None, std_layer=None)
            for i in range(len(W) - 1):
                mean_layer[i] = norm_coef *  mean_layer[i] + (1 - norm_coef) * mean_layer_out[i]
                std_layer[i] = norm_coef *  std_layer[i] + (1 - norm_coef) * std_layer_out[i]
            for i in range(len(W)):
                W_momentum[i] = W_momentum[i] * rho +  eta * grad_W[i]
                b_momentum[i] = b_momentum[i] * rho +  eta * grad_b[i]
                W[i] -= W_momentum[i]
                b[i] -= b_momentum[i]
            
    
        #print("Accurency on the test = "  + str(compute_Accuracy(X_val, y_val, W1, b1, W2, b2)))
        eta *= 0.95
        lostTrain.append(compute_Cost_batch(X, Y, W, b, lamb, eps, mean_layer, std_layer))
        lostVal.append(compute_Cost_batch(X_val, Y_val, W, b, lamb,eps, mean_layer, std_layer))
    
    return W, b, lostTrain, lostVal, mean_layer, std_layer

In [369]:
N = 10000
N_bis = 9000
K = 10
d = 3072
lamb = 1

dim_list = [d, 50, K]
X = X_train[:,:N_bis]
Y = Y_train[:,:N_bis]
y = y_train[:N_bis]

X_val = X_train[:,N_bis:]
Y_val = Y_train[:,N_bis:]
y_val = y_train[N_bis:]


n_batch= 100
eta= 0.01
n_epochs= 5
rho = 0.9
W_init, b_init = initializeWb_Hu(dim_list)
eps = 10e-06
print(W_init[0].shape)
print(compute_Cost_batch(X, Y, W_init, b_init, lamb,eps, None, None))
print(W_init[0].shape)
W, b, lostTrain, lostVal, mean_layer, std_layer = fit(X, Y, y, X_val, Y_val, y_val,  lamb, n_batch, rho, eta, n_epochs, dim_list, W_init, b_init, eps, 0.9)

print("Accurency on the test = "  + str(compute_Accuracy_batch(X_test, y_test, W, b, eps, mean_layer, std_layer)))
plot_curve(lostTrain, lostVal)

(50, 3072)
6261.054867000741
(50, 3072)
(50, 3072)
(50, 1)
j 0
(50, 3072)
[[ -5.3927631  -12.92601753 -16.09839006 ...  -7.86558571  -8.04834793
   -6.98957956]
 [-10.12496842 -10.36238966 -10.47982904 ... -13.42780786  -9.19623965
   -5.64500465]
 [  1.74481338   2.60517613   5.159706   ...   4.44739827   3.38768928
    2.8023769 ]
 ...
 [  1.25116503   2.46917663   4.02869836 ...   0.84178926   1.30899796
   -3.54902262]
 [ -0.24692414  -2.65893861  -6.33807498 ...  -1.44650468  -3.10967696
   -1.0497437 ]
 [  0.46845644  -3.7678704    2.14473209 ...  -1.45760971  -1.86226212
   -0.89628707]]
[array([ -9.35939626, -10.17967869,   3.97193435,   5.47473238,
         1.90514249,  -1.25578804,   6.3701574 ,   1.56112871,
        -2.53242901,  -0.02466872,  -2.22653978,   0.08903954,
         6.36399711,   5.36357642,  -4.40384424,  -5.78639415,
         3.66692146,   2.99143764,  -5.78532425,   2.96855765,
        -9.25923465,  -8.58961413,  -1.17402286,  -4.75367658,
        -2.4749522 

TypeError: 'NoneType' object is not subscriptable

In [231]:
N = 10000
N_bis = 1000
K = 10
d = 3072
lamb = 0
eps = 10e-06
dim_list = [d,10, K]
W, b = initializeWb_Random(dim_list)
X = X_train[:,:N_bis]
Y = Y_train[:,:N_bis]
y = y_train[:N_bis]
X_val = X_train[:,N_bis:]
Y_val = Y_train[:,N_bis:]
y_val = y_train[N_bis:]

n_batch= 100
eta= 0.0252
n_epochs= 10
rho = 0.9
print("Accurency on the test = "  + str(compute_Accuracy_batch(X_train, y_train, W, b, eps)))
print("Cost function = " + str(compute_Cost_batch(X, Y, W, b, lamb, eps)))
print("One proba : " + str(evaluate_Classifier_batch(X, W, b, eps)[0][:]))

Accurency on the test = 0.0898
Cost function = 2.3704239139224574
One proba : [[0.09520598 0.08502072 0.05515778 ... 0.08454912 0.06930327 0.06572916]
 [0.1307071  0.10956342 0.11762471 ... 0.12731229 0.14743798 0.08838231]
 [0.07880121 0.08726361 0.11994545 ... 0.09882183 0.12510053 0.06707362]
 ...
 [0.13037742 0.14061573 0.21344733 ... 0.15510613 0.19945646 0.14954028]
 [0.10591839 0.10555981 0.08052146 ... 0.09205797 0.06701409 0.11996603]
 [0.0981124  0.083172   0.06707408 ... 0.08261487 0.0567159  0.07901497]]
